# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!

## Please note:

When you run the pip installs in the first cell below, you might get this error - it can be safely ignored - it sounds quite severe, but it doesn't seem to affect anything else in this project!


> ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.



In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 8.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [2]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [3]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [6]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive",force_remount=True)
audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"

Mounted at /content/drive


# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [7]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [8]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [9]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism and you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United Sta

In [10]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [11]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [29]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings b

In [30]:
response = tokenizer.decode(outputs[0])

In [31]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
and kind of the confluence of this whole idea of the confluence week, the merging of two rivers and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. So that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism and you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous People's Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America, and to the republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. All right. Thank you, Councilman Lopez. Madam Secretary, roll call. Clerk. Here. Espinosa. Here. Flynn. Here. Gilmour. Here. Here. Cashman. Here. Kenneche. Here. Lopez. Here. New. Here. Ortega. Here. Sussman. Here. Mr. President. Here. 11 present. 11 members present. We do have a quorum. Approval of the minutes. Seeing none. Minutes of October 2nd stand approved. Council announcements. Are there any announcements by members of Council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock p.m. It will move along Broadway from 3rd to Alameda. It's going to be a fun, family-friendly event. Everyone's invited to come down, wear a costume. There will be candy for the kids and there are tiki zombies and 29 hearses and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at 6 o'clock for the Broadway Halloween parade. Thank you, Mr. President. All right. Thank you, Councilman Clark. I will be there. All right. Presentations. Madam Secretary, do we have any presentations? None, Mr. President. Communications. Do we have any communications? None, Mr. President. We do have one proclamation this evening. Proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President, with pride. Proclamation number 17, well, let me just say this differently. Proclamation number 1127, series of 2017, an observance of the second annual Indigenous Peoples Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the Indigenous Peoples have lived and flourished on the lands known as the Americas since time immemorial and that Denver and the surrounding communities are built upon the ancestral homelands of numerous Indigenous tribes, which include the Southern Ute, the Ute Mountain, Ute tribes of Colorado. And whereas the tribal homelands and seasonal encampments of the Arapaho and Cheyenne people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes and the City and County of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas on October 3, 2016, the City and County of Denver unanimously passed Council Bill 801, series of 2016, officially designating the second Monday of October of each year as Indigenous Peoples Day in Denver, Colorado. And whereas the Council of the City and County of Denver continues to recognize and value the vast contributions made to the community through Indigenous Peoples' knowledge, science, philosophy, arts, and culture. And through these contributions, the City of Denver has developed and thrived. Whereas the Indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of Indigenous people, including Confluence Week, drawing record attendance to a National Indigenous Youth Leadership Conference, leading conversations on inclusion with their peers, and supporting increased Indigenous youth participation in science and engineering. Now, therefore, be it proclaimed by the Council of the City and County of Denver, Section 1, that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of Indigenous people to our history, our past, our present, and future, and continues to promote the education of the Denver community about these historical and contemporary contributions of Indigenous people. Section 2, that the City and County of Denver, Colorado, does hereby observe October 9, 2017 as Indigenous Peoples Day. Section 3, that the Clerk of the City and County of Denver shall attest and affix the seal of the City and County of Denver to this proclamation, and that a copy be transmitted to the Denver American Indian Commission, the City and County of Denver School District No. 1, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez. Your motion to adopt. Mr. President, I move that Proclamation No. 1127, Series of 2017, be adopted. All right. It has been moved and seconded. It comes from the members of Council. Councilman Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver, officially for the second time. It's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our Council Chambers. It was a very beautiful piece of artwork that you presented to us earlier, and it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous Peoples Day. And we sat down and wrote it, and as a community, we couldn't think of anything else to begin except for the confluence of the two rivers. And those confluence of the two rivers created such a great city. And we live in such an amazing city, and we're all proud of it, and sometimes we, and a lot of people from all over the country or all over the world are proud of it, and sometimes a little too proud of it is to tell them to go back home. But I'm kidding when I say that. But the really nice thing about this is that we are celebrating Indigenous Peoples Day out of pride for who we are, who we are as a city, and the contributions of Indigenous Peoples to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt or disrespect. I think of a quote that Cesar Chavez made very popular, and it stuck with me for a very long time, and any time I have the opportunity to speak in front of children, and especially children in our community, that they often second-guess themselves on where they're coming from, who they are. And I always say that it's very important to be proud of where you're from, and the quote that I use from Cesar Chavez is, you know, pride in one's own culture does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that no matter who we are, where we come from in this society, your pride in your own culture does not require the contempt or disrespect of another. And man, what a year for that to just sit on our shoulders for a while, for us to think about, right? And so I wanted to just thank you all. I think the Commission, there's going to be a couple individuals that are going to come and speak. Thank you for your art, your lovely artwork, for us to see what's in your heart and what now has become, probably is going to be a very important symbol for the community. And also just for the work, the daily work, every single day. We still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, right, in poverty, without access to services, right, without access to sobriety or even housing or jobs. And what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do. And these kind of proclamations and this day is not a day off, it's a day on in Denver, right, in addressing those critical issues. So I know that my colleagues are very supportive. I'm going to ask you to support this proclamation, as I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again today as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Martega? Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was and had the benefit of being able to go down to the Four Corners for a peace treaty signing ceremony between the Utes and the Comanches that had been sort of at odds with each other for about 100 years, and just being able to participate in that powwow was pretty awesome. And for those of you who continue to participate in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on as well as the Native language from generation to generation is just so incredible, because in so many cultures, you know, people have come here and assimilated to the, you know, the norms here, and they lose their language and lose a lot of the culture. And in the Native community, that hasn't happened. That, you know, commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right. Thank you, Councilwoman Ortega. Councilwoman Knitsch. Thank you very much, and I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is, you know, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture, and so I think the art is so important. And when you talked about water, I was also thinking about land, and I guess I just wanted to say thank you. Many of the Native American peoples of Colorado have been at the forefront, or actually nationally, of defending some of the public lands that have been protected over the last few years that are under attack right now. And there are places that the communities have fought to protect but that everyone gets to enjoy, and so I just think that it's an example of where cultural preservation intersects with environmental protection, with recreation and all of the other ways that public lands are so important. And so I think I just wanted to say thank you for that, because I think we have some very sacred places in our country that are at risk right now. And so as we celebrate, I appreciate that there's still a piece of resistance in here, and I think that I just want to mention a feeling of solidarity with that resistance. So thank you, and happy Confluence Week. Thank you, Councilwoman Kinneach. And seeing no other comments, I'll just say a couple. And in a time of such divisive ugliness and just despicable behavior from our leadership, the reason I'm so supportive of Indigenous Peoples' Day is because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told. And so we celebrate inclusivity in the face of such evil times, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Denver City Council Meeting**
=============================================

**Summary**
-----------

The Denver City Council meeting of October 9th, 2017, was held at the Council Chambers. The meeting was attended by 11 council members, with a quorum present. The main discussion points were the proclamation of Indigenous Peoples Day in the City and County of Denver.

**Attendees**
------------

*   Councilman Lopez
*   Councilman Clark
*   Councilman Martega
*   Councilwoman Ortega
*   Councilwoman Kinneche

**Location and Date**
---------------------

*   Council Chambers
*   Monday, October 9th, 2017

**Discussion Points**
---------------------

*   Proclamation of Indigenous Peoples Day in the City and County of Denver
*   Celebration of Indigenous Peoples' contributions to the city's history, present, and future
*   Recognition of the cultural and foundational contributions of Indigenous people
*   Importance of inclusivity and respecting all cultures

**Takeaways**
------------

*   The city recognizes the importance of Indigenous Peoples' Day and its significance in promoting inclusivity and respecting all cultures.
*   The proclamation highlights the contributions of Indigenous people to the city's history, present, and future.
*   The city continues to value and celebrate the cultural and foundational contributions of Indigenous people.

**Action Items**
----------------

### Adopt Proclamation No. 1127, Series of 2017

*   **Owner:** Councilman Lopez
*   **Description:** The proclamation of Indigenous Peoples Day in the City and County of Denver.
*   **Status:** Adopted

### Support Indigenous Peoples Day Celebrations

*   **Owner:** Councilman Lopez
*   **Description:** To support the celebrations of Indigenous Peoples Day and its significance in promoting inclusivity and respecting all cultures.
*   **Status:** Supported by Council members

### Promote Cultural Preservation and Environmental Protection

*   **Owner:** Councilwoman Kinneche
*   **Description:** To promote cultural preservation and environmental protection, especially in regards to public lands.
*   **Status:** Supported by Council members<|eot_id|>

## Alternative implementation

we use an open-source model to transcribe the meeting Audio.



In [35]:
# 📦 Import required libraries
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
import IPython.display as ipd
import os
import tempfile
import gradio as gr

# 🎙️ Load Whisper model
AUDIO_MODEL = "openai/whisper-medium"

speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    AUDIO_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_safetensors=True
).to("cuda")

processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device="cuda"
)

# 🔊 Transcription Function (for Gradio)
def transcribe(audio):
    """Takes audio input, returns transcription with timestamps."""
    result = pipe(audio, return_timestamps=True)
    return result['text']

# 🎛️ Gradio Interface: Upload or Record Audio (Radio Feature)
gr.Interface(
    fn=transcribe,

    inputs=gr.Audio(type="filepath", label="🎙️ Upload or Record Audio"),

    outputs="text",
    title="📻 Whisper Radio",
    description="Talk or upload your audio — Whisper will transcribe it like a real-time radio caption feed.",
    live=True
).launch()


Device set to use cuda


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc4c5e253d6b3856e2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [33]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [24]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch

AUDIO_MODEL = "openai/whisper-medium"

# Load model and processor
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    AUDIO_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_safetensors=True
).to("cuda")

processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

# Create pipeline (no return_timestamps here)
pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device="cuda"
)

Device set to use cuda


In [27]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename,return_timestamps=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [28]:
transcription = result["text"]
print(transcription)

 and kind of the confluence of this whole idea of the Confluence Week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue. So that is the reason that the back of the logo is considered water. So let me see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the U